In [69]:
from utils import prepare_jupyter
prepare_jupyter()

import os
import pandas as pd
import numpy as np

from thesis.io import read_csv, NAMES

DATA_DIR_BASE = '/Users/tomek/University/MgrThesis/EnsembleDiversityResults'
base_path = os.path.join(DATA_DIR_BASE, 'diversity-22-09')

COLUMNS = [
    # STRUCT.
    'node_diversity',    
    'used_attributes_ratio',
    'avg_node_count',
    'avg_attributes_used',
    # BEHAV.
    'corr',
    'df',
    'entropy',
    'kw',
    'q',
    'coverage_minmax',
    'coverage_std'
]

In [70]:
def save_table_csv(table_df, name, pretty=False):
    table_path = f'/Users/tomek/University/MgrThesis/EnsembleDiversityResults/tables-15-09/{name}.csv'
    table_to_save = table_df.copy()
    
    if pretty:
        index_name_map = {
            'dataset_name': 'Dataset'
        }
        
        column_name_map = {
            'used_attributes_ratio': 'Used attr. ratio',
            'node_diversity': 'Node diversity',
            'corr': 'Correlation',
            'entropy': 'Entropy',
            'kw': 'Kohavi-Wolpert variance',
            'q': 'Q-statistic',
            'coverage_minmax': 'Coverage (minmax)',
            'coverage_std': 'Coverage (std. dev.)'
        }
        
        table_to_save = table_to_save.rename(columns=column_name_map).rename(index=index_name_map)
        
    table_to_save.to_csv(table_path)

def get_table(df, ensemble_name):
    results = {}
    
    groups = df[df['name'] == ensemble_name].groupby(by=['dataset_name'])   
    
    for group_id, group_df in groups:
        dataset_name = group_id
        
        if dataset_name not in results:
            results[dataset_name] = []
        
        for column in COLUMNS:
            values = group_df[[column, 'accuracy']]
            corr_coeff = values.corr().iat[0, 1] # Or [1, 0] - doesn't matter
            results[dataset_name].append(corr_coeff)    
    
    table = np.array([[dataset_name, *results] for dataset_name, results in results.items()])
    table_df = pd.DataFrame(table, columns=['dataset_name', *COLUMNS]).set_index('dataset_name').astype(np.float32)
    
    return table_df.round(3)

## Tabele (średnie)

1. Średnie wyniki dla każdej miary (wiersz to zbiór danych, kolumna to nazwa klas.)
2. ...

In [71]:
def df_from_groups(groups, with_avg=True):
    column_set = ['index']
    results = {}
    
    for (dataset_name, name), value in groups.items():
        if name not in column_set:
            column_set.append(name)
        
        if dataset_name not in results:
            results[dataset_name] = []
        
        results[dataset_name].append(value)
    
    
    results = np.array([[dataset_name, *values] for dataset_name, values in results.items()])

    if with_avg:
        column_avgs = np.array([['AVERAGE', *np.mean(results[:,1:].astype(np.float), axis=0)]])
        results = np.concatenate((results, column_avgs), axis=0)

    return pd.DataFrame(results, columns=column_set)


def get_avg_table(measure):
    df_path = os.path.join(base_path, 'allexperiment-ensemble.csv')
    all_df = pd.read_csv(df_path, index_col=0)
    all_df.head(n=2)

    groups = all_df.groupby(by=['dataset_name', 'name'])
    results = {}

    for group_name, group_df in groups:
        values = group_df[measure].values
        results[group_name] = np.mean(values)

    return df_from_groups(results)

In [72]:
get_avg_table('node_diversity')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,6.705776423704727,4.534673144740658,13.029224771973004,1.190220073003889,4.893634637072217,3.1774274731125605
1,boston,6.14710119515433,4.137565972484585,0.8830720033424524,3.818990024633795,5.140086232543834,5.110744124972201
2,cmc,7.714195013533241,4.709819881488884,2.8179870259603517,3.830279488071824,7.419626898959596,6.218113430634265
3,flags,7.247158154581975,5.256902613743185,2.8903229547834433,0.8184039271416983,6.774610358779737,3.3278104642073782
4,glass,5.5996130820675925,3.9671679572186402,0.664822998006925,1.0905561810096067,4.8239531665074855,3.2719249913059003
5,ionosphere,6.186434061579411,3.7207108371457314,0.0,1.9910277672351853,5.986341781702702,3.3326352444911294
6,isolet,0.0,1.751915856308536,0.3339817970707536,2.839737188445424,5.750324560316694,1.4665012864301865
7,lymph,4.881995514979682,3.766367265648041,11.404581724794562,0.940017570847456,5.2952969972552255,2.7904387444559604
8,oil_spill,4.642181203252159,3.533263480428248,0.0,3.6928211762817917,5.065862001083143,4.624994878667559
9,pollution,0.0,2.1520913292139796,5.163451618620398,0.0,2.929255818514028,1.6184070516204945


In [73]:
get_avg_table('used_attributes_ratio')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.9939393939393941,0.9915151515151517,0.9724799010513295,0.8484848484848484,0.9990909090909091,0.9
1,boston,0.981025641025641,0.9861538461538463,0.9784830554061325,0.8923076923076922,0.9948717948717949,0.923076923076923
2,cmc,1.0,0.9977777777777778,1.0,0.962962962962963,1.0,0.9851851851851852
3,flags,0.974047619047619,0.9623809523809522,0.9014084507042256,0.6142857142857142,0.9840476190476191,0.8238095238095238
4,glass,0.9992592592592593,0.9955555555555554,1.0,0.9555555555555556,0.9992592592592593,0.9407407407407408
5,ionosphere,0.942156862745098,0.892549019607843,0.8939575830332133,0.8411764705882353,0.9533333333333331,0.8019607843137256
6,isolet,0.007671528903295517,0.11817396002160993,0.23045188199159022,0.17363587250135062,0.3566828741220962,0.044840626688276604
7,lymph,0.9785185185185185,0.9722222222222221,0.8918717375093214,0.6481481481481481,0.9925925925925928,0.7888888888888889
8,oil_spill,0.8968707482993195,0.846938775510204,0.8371511870054145,0.7551020408163265,0.9190476190476187,0.6761904761904762
9,pollution,0.3479999999999999,0.9586666666666668,0.9687645687645685,0.44888888888888884,0.9946666666666666,0.72


In [74]:
get_avg_table('corr')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.2370716355139751,0.46102115016246226,0.15548384307450322,0.3022003885968626,0.43671453961632084,0.4063373688403712
1,boston,0.1544952755258015,0.4526140800778277,0.21072251529201946,0.26141482659597426,0.33023689345229185,0.31864289898975784
2,cmc,0.04473368555792962,0.5385472668907901,0.11326090662080358,0.28479279917292816,0.4298318073714657,0.33734762895495524
3,flags,0.1250265127757344,0.2889407392858981,0.05807904595241448,0.22403971170691658,0.21877131018853163,0.29714229033029715
4,glass,0.15390528540845963,0.38259986107743615,0.1667075686195846,0.17778019635239672,0.3026220362688421,0.31631344457485
5,ionosphere,0.18115946375611358,0.49837457031215765,0.11832856075967293,0.24479382500327848,0.32501315941067543,0.3583747680441128
6,isolet,0.0,0.48618192337491717,0.06955517924383087,0.41076458126073095,0.24281887742987887,0.40913432197261546
7,lymph,0.16937647142450696,0.3120709336117052,0.18196959048418673,0.19650621778931737,0.2122678570596307,0.2782320518477246
8,oil_spill,0.27854032275173496,0.5960026651497565,0.1303179510187209,0.2587339536624295,0.5009820466187567,0.48506295841750463
9,pollution,0.0,0.25987793032105677,0.17575745076917798,0.2276471655497441,0.1921451396459247,0.3912851096370245


In [75]:
get_avg_table('df')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.15247952309837104,0.17111410611580322,0.16385942245112264,0.16891790853096636,0.16933961937625705,0.1690819837157064
1,boston,0.11865744023248528,0.10822673585219,0.10875967664038066,0.1250584711398978,0.1103163080667144,0.12737976267948256
2,cmc,0.2175205081474443,0.28598663622065384,0.22271470766161186,0.21951490833938492,0.2957999106936815,0.24783375357314838
3,flags,0.19492352839771385,0.18593530551876414,0.1962337545573805,0.20414381905108722,0.18557507178609933,0.21021086831111893
4,glass,0.119554687730546,0.14130843287973488,0.14050399502694763,0.1616416816556035,0.13173121329384577,0.14583770460878512
5,ionosphere,0.07299838455844894,0.0744457542865936,0.08928081244073993,0.08082672581907994,0.06435572376055274,0.08097294609407285
6,isolet,0.0,0.01927094757094757,0.05792869900662109,0.018429320415034704,0.01595816074577979,0.026943988639226736
7,lymph,0.09629071106533871,0.10117307345090594,0.10325272190184413,0.13951782710994534,0.10598354513940722,0.10486279196689706
8,oil_spill,0.04852040012396055,0.04289085998934674,0.07798045600445368,0.09697840597661803,0.042984743853581524,0.05247210416511328
9,pollution,0.0,0.13787659818136005,0.13994587663418834,0.15637027875123113,0.139100300052681,0.18995645800407707


In [76]:
get_avg_table('entropy')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.5067135336594455,0.29486470999301184,0.5656981227151936,0.4544523643139996,0.2974913580246913,0.35893221523410207
1,boston,0.5131855044327962,0.2503599948230117,0.45543443937153893,0.45349343169611084,0.3345296317867081,0.40033184494920077
2,cmc,0.8255647457627118,0.2764811222568124,0.6534722188463109,0.5104009070294785,0.3520075022099235,0.45729168684422916
3,flags,0.6398687359424201,0.47241997300944677,0.7236849708235921,0.5586900584795322,0.5326940170940172,0.4939208277103014
4,glass,0.5024826135105205,0.329162200073828,0.5313505906238465,0.5564584717607973,0.3789111111111111,0.41525876707272064
5,ionosphere,0.3578431015723974,0.16068303152246816,0.4758739101274313,0.3601196512407781,0.22504236083165666,0.2910331321260899
6,isolet,0.0,0.04411222222222221,0.4586678321678321,0.07848888888888889,0.09366,0.1312333333333333
7,lymph,0.4412895469912102,0.32914452107279696,0.4534151045282727,0.51832030651341,0.42563754789272035,0.38309885057471266
8,oil_spill,0.25905779213519775,0.09458075624834074,0.5042202567680845,0.392473508552357,0.14591886069708346,0.19193666325330905
9,pollution,0.0,0.4424444444444444,0.5307925407925408,0.5177777777777777,0.5009222222222222,0.3897777777777777


In [77]:
get_avg_table('kw')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.15997179359961114,0.10372007313516242,0.18909138426938357,0.1831846880568582,0.10721649189855001,0.16117776749011892
1,boston,0.09759200573534121,0.04679857075838569,0.09030212512577021,0.0969204197287727,0.06396979843678521,0.08911835049002179
2,cmc,0.0430782520986236,0.01774979019641373,0.03927485381560927,0.035696392071148185,0.02248312570428371,0.03382358264457666
3,flags,0.29124080657802165,0.21825628738443564,0.30997024945728135,0.27701164369798503,0.24542855765664204,0.26361274050033906
4,glass,0.22366709616482855,0.14674298574907396,0.22380515619904992,0.2514042292477354,0.16889438856246783,0.2116627238600506
5,ionosphere,0.09547105983890225,0.044789892686771186,0.1282569856070958,0.11484655080044318,0.06377016614509323,0.10348273258599214
6,isolet,0.0,0.00799089351851852,0.07783176476301477,0.019133425925925925,0.017213740740740745,0.03603541666666667
7,lymph,0.28767550933728536,0.21140471836878494,0.2824202039225701,0.3430357739903995,0.27046163341700796,0.2857709481657638
8,oil_spill,0.02985450675760454,0.00996046927914937,0.05300744929429284,0.04478045914790562,0.016626558665880458,0.02849497815355767
9,pollution,0.0,0.6661842592592592,0.7909916472416472,0.8052777777777778,0.7592087962962963,0.7192037037037037


In [78]:
get_avg_table('coverage_minmax')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.06549110855795237,0.0472898395203418,0.07369122706857839,0.03271696208013766,0.047697427832092645,0.07484717626179366
1,boston,0.0795160716244479,0.05875267654606124,0.12281359110438805,0.09874278072377363,0.0792368999820726,0.06393375181109343
2,cmc,0.20897878767004424,0.07182608699337116,0.14382580843319265,0.1368348629938433,0.0938882948117366,0.1502997175025009
3,flags,0.06770964069191013,0.052162500739614884,0.12641468569341435,0.017556712956301877,0.0734475943041327,0.03743262308970824
4,glass,0.07347795989082175,0.08795393971328257,0.11044793908921798,0.028053536690408953,0.08577921312133995,0.07647793530985442
5,ionosphere,0.09967238138551672,0.05085130937627566,0.10249613134384167,0.08101660108919395,0.10119051973148084,0.09714941977253128
6,isolet,0.0,0.019717313831703446,0.16998498755782926,0.03739846597757889,0.036780249731143286,0.07165901012998525
7,lymph,0.0636535800069113,0.04780896346345535,0.08675271210996847,0.028130040910935166,0.06340673885100419,0.06248567344022766
8,oil_spill,0.1439637527356607,0.053762110483521246,0.14527933864404924,0.10724771335358732,0.13872241927133802,0.11571677928072939
9,pollution,0.0,0.060143441467550636,0.08481483137993852,0.04870026654725171,0.07307877814097748,0.06770646363613074


In [79]:
get_avg_table('coverage_std')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.01773263965052862,0.01342700762330838,0.028662765519259087,0.01091083820143434,0.012981608063351519,0.02531389932481923
1,boston,0.01659651109655412,0.013734562111249099,0.018891712960759314,0.03217009026809991,0.01658590825426458,0.017163316962902304
2,cmc,0.05419131850698408,0.014387520192749591,0.025670098535559002,0.038677862904681115,0.01829918147976616,0.03817379047359953
3,flags,0.016207854971850438,0.014325850074966971,0.02504613118701655,0.007214815837071023,0.020799171399906755,0.013428935194039643
4,glass,0.015233657191885068,0.019084618664794963,0.016278023317821372,0.010517550649379247,0.020157347176319076,0.019233545776094332
5,ionosphere,0.025938886946433162,0.02398029238606728,0.014678995382938419,0.028543874221646087,0.03726760676599695,0.03091857795966813
6,isolet,0.0,0.012561259877809702,0.027125697544343765,0.022313133075702193,0.032496508256191305,0.036999106375908905
7,lymph,0.013793273822259105,0.013341083102725441,0.025291020071178096,0.012859560026325384,0.016240286487649462,0.019338898555331017
8,oil_spill,0.03014396670382343,0.02250626671882941,0.020882626057741365,0.03131110492654272,0.03591970763472884,0.039526449773695266
9,pollution,0.0,0.02576951377417322,0.014306324667936422,0.024350133273625854,0.02833903902124894,0.02822444895960765


In [81]:
get_avg_table('q')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.3673305334842701,0.7695914030729523,0.2782074168294673,0.4677788381698912,0.7107276108660457,0.604620692484301
1,boston,0.314901607372291,0.7817238615608364,0.3827041996445249,0.4224434613155889,0.6130991728422809,0.5394049557927716
2,cmc,0.08544224529566201,0.8369429969630011,0.20719497434642514,0.4707274096221905,0.7263169310706608,0.5333924011416888
3,flags,0.23289750351209032,0.5112606108466289,0.1028646466176554,0.3527944116340086,0.3974841581125743,0.4723538785732323
4,glass,0.2969922381803129,0.6668704717110607,0.2982158272858107,0.2588957979695125,0.5543521973256504,0.5162845806921242
5,ionosphere,0.38071232722055487,0.8533095906735609,0.22574504811045143,0.4366866031876583,0.640489400840688,0.58317590450929
6,isolet,0.0,0.7725948888609842,0.08617783994152121,0.5601247803498475,0.40892158860814865,0.4808212826843237
7,lymph,0.3011783782537182,0.5549259151167515,0.30387557671843113,0.2579037414490966,0.3877007245897594,0.41428323614205237
8,oil_spill,0.6010492559199861,0.9094986908196477,0.2687120779340799,0.480525892281775,0.8129177958901631,0.7631052636596193
9,pollution,0.0,0.33354820549682224,0.24368846921691595,0.1866560938318722,0.2552468428504131,0.47858948587596617


In [82]:
get_avg_table('accuracy')

,index,Adaboost,Bagging,CatboostEnsemble,LightGBM,RandomForest,XGBoostEnsemble
0,SPECT,0.7812834847426042,0.7607337526205451,0.7930375504499494,0.7783135336594456,0.7406335895644071,0.7868623340321453
1,boston,0.8495521905131689,0.8512062382708857,0.8573527035961513,0.8458228175758751,0.8406736556008542,0.8491555037856726
2,cmc,0.623061147622891,0.636987509127945,0.6365461622120219,0.6407317729351628,0.6162404396786962,0.6549913524731926
3,flags,0.666378767431399,0.7038956365272154,0.6820959495162608,0.6615384615384615,0.6917318938371569,0.6631129104813315
4,glass,0.8301365817644886,0.7890143964562568,0.8163913498215822,0.8133997785160576,0.8143816906607604,0.8008121077888518
5,ionosphere,0.9110301810865191,0.9044225352112674,0.9093650337398541,0.9127967806841047,0.9189577464788734,0.9013816230717638
6,isolet,0.9681666666666668,0.9745555555555556,0.9782051282051282,0.9799999999999998,0.9789444444444443,0.9772222222222221
7,lymph,0.8811877394636015,0.8443908045977011,0.8407004551415568,0.837624521072797,0.8475402298850575,0.8734099616858239
8,oil_spill,0.936077103955702,0.9407511283043197,0.942913722548981,0.9420127432017296,0.9495399552470892,0.9341942579739828
9,pollution,0.681111111111111,0.7861111111111111,0.793123543123543,0.75,0.7794444444444444,0.7555555555555554
